# Calculate the polarizability for two water molecules the fast and simple way

The tutorial https://github.com/fishstamp82/moltools/blob/docs/share/tutorial/two_water_example.ipynb  
demonstrates the step-by-step process in how the Applequist equations are solved from quantum mechanical properties.

---------

**This** tutorial will perform all the above steps in a more compact, and faster way, using the moltools package.

----------

## DALTON
In this tutorial, we will be running DALTON from inside python. For this we will utilize the full path to the DALTON runscript. The location will be '/home/user/repos/dalton/build_gnu/dalton', but is of course custom to every user.


In [29]:
DALTON = '/home/user/repos/dalton/build_gnu/dalton'

## Moltools

Clone the entire repository, which will come with loprop and pd automatically:
    
```bash
$> git clone --recursive https://github.com/fishstamp82/moltools.git
```

(*Optional, but **highly recommended***) If you want optimized parallel excecution, make sure you have Cython and build the cython extensions of vahtras/pd by running the following script:

```bash
$> moltools/scripts/build_cython.sh
```


**Make sure that the path to moltools is in your PYTHONPATH env variable**, and run an ipython console

```bash
$> ipython
```

In [30]:
from moltools import *

The get_standard method creates a Water instance, with the geometry of a TIP3P water model. It's oxygen is placed in origo, the molecule in the xz-plane, with it' electronic dipole pointing in the positive z-axis

In [31]:
w = Water.get_standard()

Take a look at it

In [32]:
#w.plot()

the molecular property of this water molecule is accesible via *Water*.Property or *Water*.p

In [33]:
print w.p

{'beta': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 'alpha': array([ 0.,  0.,  0.,  0.,  0.,  0.]), 'charge': 0.0, 'dipole': array([ 0.,  0.,  0.]), 'quadrupole': array([ 0.,  0.,  0.,  0.,  0.,  0.])}


All properties are zero. Let's obtain properties via QM using DALTON.


In [34]:
DALTON = '/home/ignat/repos/dalton/build_gnu/dalton'


w.props_from_qm( method = 'hfqua', #hf is Hartree-Fock, qua is Quadratic response (LoProp Beta)
                dalpath = DALTON, 
                tmpdir = '/tmp',
                basis = ['6-31+G*'],
               )

Now the properties are non-zero

In [35]:
print "Charge of the water molecule:" , w.p.q
print "Dipole moment of oxygen:" , w.o.p.q

print "Dipole moment of the water molecule:" , w.p.d
print "Dipole moment of oxygen:" , w.o.p.d

Charge of the water molecule: -1.00000000003e-07
Dipole moment of oxygen: -0.1482071
Dipole moment of the water molecule: [ 0.          0.          0.91646495]
Dipole moment of oxygen: [ 0.         0.         0.7925135]


### Now it is straightforward to calculate the properties of individual molecules.

We now create an additional water molecule, translate it to (x, y, z) = (0, 0, 5 ), and calculate the Applequist beta.


In [36]:
water2 = Water.get_standard().t(0, 0, 5)
water2.props_from_qm( method = 'hfqua', #hf is Hartree-Fock, qua is Quadratic response (LoProp Beta)
                dalpath = DALTON, 
                tmpdir = '/tmp',
                basis = ['6-31+G*'],
               )

Using Cluster, we group the molecules to produce a list of Applequist point dipoles.

In [37]:
c = Cluster( w, water2 )
c.populate_bonds()

In [44]:
#Uncomment to visualize it

#c.plot()

In [42]:
#get_pdlist generates the input format file for PointDipoleList automatically based on the molecules
#in the cluster and their unit of coordinate

pdlist = c.get_pdlist()
print "Beta:", pdlist.beta( cython = 1, num_threads = 4 )

Beta: [[[ -8.88178420e-16   0.00000000e+00  -2.48498037e+01]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [ -2.48498037e+01   0.00000000e+00  -4.44089210e-16]]

 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   5.16338869e-01]
  [  0.00000000e+00   5.16338869e-01   0.00000000e+00]]

 [[ -2.48498037e+01   0.00000000e+00  -8.88178420e-16]
  [  0.00000000e+00   5.16338869e-01   0.00000000e+00]
  [ -8.88178420e-16   0.00000000e+00  -9.21686820e+00]]]
